# Predicting the S&P 500 based on Trends in Oil prices (WTI) and 10 year Treasury Bond Rates

 In this experiment, we will try to build a model that can predict the movement of S&P 500 based on corresponding price fluctuations in crude oil prices and treasury bonds interest rates. While stock market prediction is not a new endeavor, research of correlation of S&P 500 with prices of commodities (like crude oil) and indicators of inflation (like 10 year Bond rates) seems rare. 

In this exercise, we use publicly available information from the following sources to conduct this experiment: 
1. <a href="https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC"> Yahoo Finance </a> - S&P 500 historical data,
2. <a href="https://fred.stlouisfed.org/series/DGS10"> FRED Interest rates</a> - 10 Year Treasury constant maturity rate,
3. <a href="https://fred.stlouisfed.org/series/DCOILWTICO"> FRED Oil Prices</a> - WTI crude oil prices

## 1. Data Preparation and Cleanup

#### i. Load data

Let us first load the S&P 500 dataset and see if we need to do any cleanup. 

In [58]:
library("caret")
library ("e1071")
library ("klaR")
library("rpart")
library("gbm")

Loading required package: lattice
Loading required package: ggplot2
Loading required package: MASS
Loading required package: survival

Attaching package: ‘survival’

The following object is masked from ‘package:caret’:

    cluster

Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3


In [10]:
snp500 <- read.csv('s&p500_1990_2017.csv',skip = 0, header = T, as.is = T)

In [11]:
summary(snp500)

     Date                Open             High             Low        
 Length:6881        Min.   : 295.4   Min.   : 301.4   Min.   : 294.5  
 Class :character   1st Qu.: 700.9   1st Qu.: 704.4   1st Qu.: 697.2  
 Mode  :character   Median :1144.0   Median :1150.6   Median :1135.9  
                    Mean   :1120.8   Mean   :1127.6   Mean   :1113.6  
                    3rd Qu.:1392.9   3rd Qu.:1401.8   3rd Qu.:1381.4  
                    Max.   :2394.8   Max.   :2401.0   Max.   :2380.2  
     Close            Volume            Adj.Close     
 Min.   : 295.5   Min.   :1.499e+07   Min.   : 295.5  
 1st Qu.: 700.9   1st Qu.:4.476e+08   1st Qu.: 700.9  
 Median :1144.1   Median :1.439e+09   Median :1144.1  
 Mean   :1121.1   Mean   :2.055e+09   Mean   :1121.1  
 3rd Qu.:1393.6   3rd Qu.:3.462e+09   3rd Qu.:1393.6  
 Max.   :2396.0   Max.   :1.146e+10   Max.   :2396.0  

Observations: 
1. The date field is character type. We should convert it to Date. 
2. Since we are only interested with the closing price, we just need to consider the Adjusted close value (as this price considers any corporate actions as well). 
3. The distribution of the adjusted close price value ranges from 295.5 to 2396 over a period of about 27 years. Woah!

In [22]:
# Change date to Date
snp500$Date <- as.Date(snp500df$Date, '%Y-%m-%d')

# Select only date and adjusted close value columns
snp500 <- snp500[,c('Date', 'Adj.Close')]
colnames(snp500)[2] <- 'snp_adj_close'

head(snp500)

Date,snp_adj_close
2017-04-21,2348.69
2017-04-20,2355.84
2017-04-19,2338.17
2017-04-18,2342.19
2017-04-17,2349.01
2017-04-13,2328.95


Next, let us load the 10 year bond rates, and just as S&P500, we will only consider the date and adj. close columns

In [17]:
bond10yr <- read.csv('10_Yr_Bond_rates.csv')
bond10yr$Date <- as.Date(bond10yr$Date, '%Y-%m-%d')
bond10yr <- bond10yr[,c('Date', 'Adj.Close')]
colnames(bond10yr)[2] <- 'bond10yr_adj_close'
head(bond10yr)

Date,bond10yr_adj_close
2017-04-21,2.237
2017-04-20,2.241
2017-04-19,2.202
2017-04-18,2.179
2017-04-17,2.252
2017-04-13,2.232


Similarly, let us load the crude oil prices. We also rename the price column and convert it to numeric type.

In [20]:
crudeoil <- read.csv('DCOILWTICO.csv')
crudeoil$DATE <- as.Date(crudeoil$DATE, '%Y-%m-%d')
colnames(crudeoil)[2] <- 'crude_oil_price_usd'
crudeoil$crude_oil_price_usd <- as.numeric(crudeoil$crude_oil_price_usd)
head(crudeoil)

DATE,crude_oil_price_usd
1990-01-02,1262
1990-01-03,1310
1990-01-04,1293
1990-01-05,1272
1990-01-08,1156
1990-01-09,1213


#### ii. Check for missing values

In [23]:
colSums(is.na(snp500))
colSums(is.na(bond10yr))
colSums(is.na(crudeoil))

Date snp_adj_close 
            0             0

Date bond10yr_adj_close 
                 0                  0

DATE crude_oil_price_usd 
                  0                   0

No missing values. That's good. However, stock markets, commodity exchanges are not operational every day. Let us select only the common data points between the three datasets for the same dates and ignore other days. 

#### iii. Select common data points

In [28]:
library(sqldf)
merged_df <- sqldf('SELECT snp500.Date, snp_adj_close, bond10yr_adj_close, crude_oil_price_usd
                   FROM snp500 
                   INNER JOIN bond10yr ON (snp500.Date = bond10yr.Date)
                   INNER JOIN crudeoil ON (snp500.Date = crudeoil.DATE)')

head(merged_df)

Date,snp_adj_close,bond10yr_adj_close,crude_oil_price_usd
2017-04-17,2349.01,2.252,2681
2017-04-13,2328.95,2.232,2702
2017-04-12,2344.93,2.296,2699
2017-04-11,2353.78,2.298,2710
2017-04-10,2357.16,2.361,2697
2017-04-07,2355.54,2.373,2667


Finally, we have the dataset to work on. 

#### iv. Computing price change

To figure out if crude oil prices and interest rates have any correlation with S&P 500 prices, we will first calculate the price changes between the current day and one, two, three, four and five days prior. In other words, we will use t-1, t-2, t-3, t-4, t-5 prices to predict the t prices. 

If prices for t is higher than t-1, then we will encode priceDir as 'Up' i.e price direction is positive. Otherwise, priceDir will be negative ('Down'). 

In [89]:
# compute price change for snp500
snp_pc1 = c(); snp_pc2= c(); snp_pc3 = c(); snp_pc4 = c(); snp_pc5 = c(); priceDir = c();
for (i in 7:nrow(merged_df)){
  snp_pc1[i] <- (merged_df$snp_adj_close[i - 1] - merged_df$snp_adj_close[i - 2]) / (merged_df$snp_adj_close[i - 1])
  snp_pc2[i] <- (merged_df$snp_adj_close[i - 2] - merged_df$snp_adj_close[i - 3]) / (merged_df$snp_adj_close[i - 2])
  snp_pc3[i] <- (merged_df$snp_adj_close[i - 3] - merged_df$snp_adj_close[i - 4]) / (merged_df$snp_adj_close[i - 3])
  snp_pc4[i] <- (merged_df$snp_adj_close[i - 4] - merged_df$snp_adj_close[i - 5]) / (merged_df$snp_adj_close[i - 4])
  snp_pc5[i] <- (merged_df$snp_adj_close[i - 5] - merged_df$snp_adj_close[i - 6]) / (merged_df$snp_adj_close[i - 5])
  priceDir[i] <- ifelse((merged_df$snp_adj_close[i] - merged_df$snp_adj_close[i - 1]) > 0, 'Up', 'Down')
}

# compute price change for bond10yr
bnd_pc1 = c(); bnd_pc2= c(); bnd_pc3 = c(); bnd_pc4 = c(); bnd_pc5 = c();
for (i in 7:nrow(merged_df)){
  bnd_pc1[i] <- (merged_df$bond10yr_adj_close[i - 1] - merged_df$bond10yr_adj_close[i - 2]) / (merged_df$bond10yr_adj_close[i - 1])
  bnd_pc2[i] <- (merged_df$bond10yr_adj_close[i - 2] - merged_df$bond10yr_adj_close[i - 3]) / (merged_df$bond10yr_adj_close[i - 2])
  bnd_pc3[i] <- (merged_df$bond10yr_adj_close[i - 3] - merged_df$bond10yr_adj_close[i - 4]) / (merged_df$bond10yr_adj_close[i - 3])
  bnd_pc4[i] <- (merged_df$bond10yr_adj_close[i - 4] - merged_df$bond10yr_adj_close[i - 5]) / (merged_df$bond10yr_adj_close[i - 4])
  bnd_pc5[i] <- (merged_df$bond10yr_adj_close[i - 5] - merged_df$bond10yr_adj_close[i - 6]) / (merged_df$bond10yr_adj_close[i - 5])
}

# compute price change for crude oil
oil_pc1 = c(); oil_pc2= c(); oil_pc3 = c(); oil_pc4 = c(); oil_pc5 = c();
for (i in 7:nrow(merged_df)){
  oil_pc1[i] <- (merged_df$crude_oil_price_usd[i - 1] - merged_df$crude_oil_price_usd[i - 2]) / (merged_df$crude_oil_price_usd[i - 1])
  oil_pc2[i] <- (merged_df$crude_oil_price_usd[i - 2] - merged_df$crude_oil_price_usd[i - 3]) / (merged_df$crude_oil_price_usd[i - 2])
  oil_pc3[i] <- (merged_df$crude_oil_price_usd[i - 3] - merged_df$crude_oil_price_usd[i - 4]) / (merged_df$crude_oil_price_usd[i - 3])
  oil_pc4[i] <- (merged_df$crude_oil_price_usd[i - 4] - merged_df$crude_oil_price_usd[i - 5]) / (merged_df$crude_oil_price_usd[i - 4])
  oil_pc5[i] <- (merged_df$crude_oil_price_usd[i - 5] - merged_df$crude_oil_price_usd[i - 6]) / (merged_df$crude_oil_price_usd[i - 5])
}

Now that we have the changes, let us create some dummy variables to answer the question - Was the change good or bad. 

We will create 5 categories based on this logic:
- Awful (Change <-1 standard deviation)
- Bad (-1 stdev <= Change < -.3 stdev)
- Unchanged( -.3 stdev <= Change < .3 stdev)
- Good (.3 stdev <= Change < 1 stdev)
- Great (Change >= 1 stdev)

In [90]:
# create categorical values for snp
snp.sd1 = sd(snp_pc1)
snp_cat1 = c(); snp_cat2= c(); snp_cat3 = c(); snp_cat4 = c(); snp_cat5 = c();
for (i in 7:nrow(merged_df)){
  
  snp_cat1[i] = ifelse(snp_pc1[i] < -1 * snp.sd, 'awful', 
                       (ifelse((snp_pc1[i] >= -1 * snp.sd && snp_pc1[i] < -0.3 * snp.sd), 'Bad',
                               (ifelse((snp_pc1[i] >= -0.3 * snp.sd && snp_pc1[i] < 0.3 * snp.sd), 'Unchanged',
                                       (ifelse((snp_pc1[i] >= 0.3 * snp.sd && snp_pc1[i] < snp.sd), 'Good',
                                               (ifelse((snp_pc1[i] >= snp.sd), 'Great', 'None')))))))))
  snp_cat2[i] = ifelse(snp_pc2[i] < -1 * snp.sd, 'awful', 
                       (ifelse((snp_pc2[i] >= -1 * snp.sd && snp_pc2[i] < -0.3 * snp.sd), 'Bad',
                               (ifelse((snp_pc2[i] >= -0.3 * snp.sd && snp_pc2[i] < 0.3 * snp.sd), 'Unchanged',
                                       (ifelse((snp_pc2[i] >= 0.3 * snp.sd && snp_pc2[i] < snp.sd), 'Good',
                                               (ifelse((snp_pc2[i] >= snp.sd), 'Great', 'None')))))))))
  
  snp_cat3[i] = ifelse(snp_pc3[i] < -1 * snp.sd, 'awful', 
                       (ifelse((snp_pc3[i] >= -1 * snp.sd && snp_pc3[i] < -0.3 * snp.sd), 'Bad',
                               (ifelse((snp_pc3[i] >= -0.3 * snp.sd && snp_pc3[i] < 0.3 * snp.sd), 'Unchanged',
                                       (ifelse((snp_pc3[i] >= 0.3 * snp.sd && snp_pc3[i] < snp.sd), 'Good',
                                               (ifelse((snp_pc3[i] >= snp.sd), 'Great', 'None')))))))))
  
  snp_cat4[i] = ifelse(snp_pc4[i] < -1 * snp.sd, 'awful', 
                       (ifelse((snp_pc4[i] >= -1 * snp.sd && snp_pc4[i] < -0.3 * snp.sd), 'Bad',
                               (ifelse((snp_pc4[i] >= -0.3 * snp.sd && snp_pc4[i] < 0.3 * snp.sd), 'Unchanged',
                                       (ifelse((snp_pc4[i] >= 0.3 * snp.sd && snp_pc4[i] < snp.sd), 'Good',
                                               (ifelse((snp_pc4[i] >= snp.sd), 'Great', 'None')))))))))
  snp_cat5[i] = ifelse(snp_pc5[i] < -1 * snp.sd, 'awful', 
                       (ifelse((snp_pc5[i] >= -1 * snp.sd && snp_pc5[i] < -0.3 * snp.sd), 'Bad',
                               (ifelse((snp_pc5[i] >= -0.3 * snp.sd && snp_pc5[i] < 0.3 * snp.sd), 'Unchanged',
                                       (ifelse((snp_pc5[i] >= 0.3 * snp.sd && snp_pc5[i] < snp.sd), 'Good',
                                               (ifelse((snp_pc5[i] >= snp.sd), 'Great', 'None')))))))))
  
  
}

# create categorical values for 10yrbond
bnd.sd = sd(merged_df$bond10yr_adj_close)
bnd_cat1 = c(); bnd_cat2= c(); bnd_cat3 = c(); bnd_cat4 = c(); bnd_cat5 = c();
for (i in 7:nrow(merged_df)){
  
  bnd_cat1[i] = ifelse(bnd_pc1[i] < -1 * bnd.sd, 'awful', 
                       (ifelse((bnd_pc1[i] >= -1 * bnd.sd && bnd_pc1[i] < -0.3 * bnd.sd), 'Bad',
                               (ifelse((bnd_pc1[i] >= -0.3 * bnd.sd && bnd_pc1[i] < 0.3 * bnd.sd), 'Unchanged',
                                       (ifelse((bnd_pc1[i] >= 0.3 * bnd.sd && bnd_pc1[i] < bnd.sd), 'Good',
                                               (ifelse((bnd_pc1[i] >= bnd.sd), 'Great', 'None')))))))))
  bnd_cat2[i] = ifelse(bnd_pc2[i] < -1 * bnd.sd, 'awful', 
                       (ifelse((bnd_pc2[i] >= -1 * bnd.sd && bnd_pc2[i] < -0.3 * bnd.sd), 'Bad',
                               (ifelse((bnd_pc2[i] >= -0.3 * bnd.sd && bnd_pc2[i] < 0.3 * bnd.sd), 'Unchanged',
                                       (ifelse((bnd_pc2[i] >= 0.3 * bnd.sd && bnd_pc2[i] < bnd.sd), 'Good',
                                               (ifelse((bnd_pc2[i] >= bnd.sd), 'Great', 'None')))))))))
  
  bnd_cat3[i] = ifelse(bnd_pc3[i] < -1 * bnd.sd, 'awful', 
                       (ifelse((bnd_pc3[i] >= -1 * bnd.sd && bnd_pc3[i] < -0.3 * bnd.sd), 'Bad',
                               (ifelse((bnd_pc3[i] >= -0.3 * bnd.sd && bnd_pc3[i] < 0.3 * bnd.sd), 'Unchanged',
                                       (ifelse((bnd_pc3[i] >= 0.3 * bnd.sd && bnd_pc3[i] < bnd.sd), 'Good',
                                               (ifelse((bnd_pc3[i] >= bnd.sd), 'Great', 'None')))))))))
  
  bnd_cat4[i] = ifelse(bnd_pc4[i] < -1 * bnd.sd, 'awful', 
                       (ifelse((bnd_pc4[i] >= -1 * bnd.sd && bnd_pc4[i] < -0.3 * bnd.sd), 'Bad',
                               (ifelse((bnd_pc4[i] >= -0.3 * bnd.sd && bnd_pc4[i] < 0.3 * bnd.sd), 'Unchanged',
                                       (ifelse((bnd_pc4[i] >= 0.3 * bnd.sd && bnd_pc4[i] < bnd.sd), 'Good',
                                               (ifelse((bnd_pc4[i] >= bnd.sd), 'Great', 'None')))))))))
  bnd_cat5[i] = ifelse(bnd_pc5[i] < -1 * bnd.sd, 'awful', 
                       (ifelse((bnd_pc5[i] >= -1 * bnd.sd && bnd_pc5[i] < -0.3 * bnd.sd), 'Bad',
                               (ifelse((bnd_pc5[i] >= -0.3 * bnd.sd && bnd_pc5[i] < 0.3 * bnd.sd), 'Unchanged',
                                       (ifelse((bnd_pc5[i] >= 0.3 * bnd.sd && bnd_pc5[i] < bnd.sd), 'Good',
                                               (ifelse((bnd_pc5[i] >= bnd.sd), 'Great', 'None')))))))))
  
}

# create categorical values for 10yrbond
oil.sd = sd(merged_df$crude_oil_price_usd)
oil_cat1 = c(); oil_cat2= c(); oil_cat3 = c(); oil_cat4 = c(); oil_cat5 = c();
for (i in 7:nrow(merged_df)){
  
  oil_cat1[i] = ifelse(oil_pc1[i] < -1 * oil.sd, 'awful', 
                       (ifelse((oil_pc1[i] >= -1 * oil.sd && oil_pc1[i] < -0.3 * oil.sd), 'Bad',
                               (ifelse((oil_pc1[i] >= -0.3 * oil.sd && oil_pc1[i] < 0.3 * oil.sd), 'Unchanged',
                                       (ifelse((oil_pc1[i] >= 0.3 * oil.sd && oil_pc1[i] < oil.sd), 'Good',
                                               (ifelse((oil_pc1[i] >= oil.sd), 'Great', 'None')))))))))
  oil_cat2[i] = ifelse(oil_pc2[i] < -1 * oil.sd, 'awful', 
                       (ifelse((oil_pc2[i] >= -1 * oil.sd && oil_pc2[i] < -0.3 * oil.sd), 'Bad',
                               (ifelse((oil_pc2[i] >= -0.3 * oil.sd && oil_pc2[i] < 0.3 * oil.sd), 'Unchanged',
                                       (ifelse((oil_pc2[i] >= 0.3 * oil.sd && oil_pc2[i] < oil.sd), 'Good',
                                               (ifelse((oil_pc2[i] >= oil.sd), 'Great', 'None')))))))))
  
  oil_cat3[i] = ifelse(oil_pc3[i] < -1 * oil.sd, 'awful', 
                       (ifelse((oil_pc3[i] >= -1 * oil.sd && oil_pc3[i] < -0.3 * oil.sd), 'Bad',
                               (ifelse((oil_pc3[i] >= -0.3 * oil.sd && oil_pc3[i] < 0.3 * oil.sd), 'Unchanged',
                                       (ifelse((oil_pc3[i] >= 0.3 * oil.sd && oil_pc3[i] < oil.sd), 'Good',
                                               (ifelse((oil_pc3[i] >= oil.sd), 'Great', 'None')))))))))
  
  oil_cat4[i] = ifelse(oil_pc4[i] < -1 * oil.sd, 'awful', 
                       (ifelse((oil_pc4[i] >= -1 * oil.sd && oil_pc4[i] < -0.3 * oil.sd), 'Bad',
                               (ifelse((oil_pc4[i] >= -0.3 * oil.sd && oil_pc4[i] < 0.3 * oil.sd), 'Unchanged',
                                       (ifelse((oil_pc4[i] >= 0.3 * oil.sd && oil_pc4[i] < oil.sd), 'Good',
                                               (ifelse((oil_pc4[i] >= oil.sd), 'Great', 'None')))))))))
  oil_cat5[i] = ifelse(oil_pc5[i] < -1 * oil.sd, 'awful', 
                       (ifelse((oil_pc5[i] >= -1 * oil.sd && oil_pc5[i] < -0.3 * oil.sd), 'Bad',
                               (ifelse((oil_pc5[i] >= -0.3 * oil.sd && oil_pc5[i] < 0.3 * oil.sd), 'Unchanged',
                                       (ifelse((oil_pc5[i] >= 0.3 * oil.sd && oil_pc5[i] < oil.sd), 'Good',
                                               (ifelse((oil_pc5[i] >= oil.sd), 'Great', 'None')))))))))
  
}

Now that we have the final categorical values to work on, let us merge all columns in a single data frame. We will also remove the first 7 rows as they will have null values. 

In [91]:
# combine all the columns for the final time and remove the top 7 rows that have NA value for categories
final_merged_df <- data.frame(priceDir, snp_cat1, snp_cat2, snp_cat3, snp_cat4, snp_cat5, bnd_cat1, bnd_cat2, bnd_cat3, bnd_cat4, bnd_cat5, oil_cat1, oil_cat2, oil_cat3, oil_cat4, oil_cat5 )
final_merged_df <- tail(final_merged_df, -6)

head(final_merged_df)

,priceDir,snp_cat1,snp_cat2,snp_cat3,snp_cat4,snp_cat5,bnd_cat1,bnd_cat2,bnd_cat3,bnd_cat4,bnd_cat5,oil_cat1,oil_cat2,oil_cat3,oil_cat4,oil_cat5
7,Up,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged
8,Down,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged
9,Up,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged
10,Down,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged
11,Up,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged
12,Up,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged,Unchanged


## 2. Build a Predictive Model

Using the dataset that we just prepared, let us build some models that can help us predict the S&P 500 based on the categories we built. We will built a 50:50% partition of train and test datasets that we will use to train and then test our model. 

In [92]:
# partition dataset into training and test
train <- final_merged_df[1:5131,]
test <- final_merged_df[5132:nrow(final_merged_df),]

#### i. Naive Bayes

In [93]:
summary(train)

 priceDir         snp_cat1         snp_cat2         snp_cat3   
 Down:2734   Unchanged:5131   Unchanged:5131   Unchanged:5131  
 Up  :2397                                                     
                                                               
      snp_cat4         snp_cat5         bnd_cat1         bnd_cat2   
 Unchanged:5131   Unchanged:5131   Unchanged:5131   Unchanged:5131  
                                                                    
                                                                    
      bnd_cat3         bnd_cat4         bnd_cat5         oil_cat1   
 Unchanged:5131   Unchanged:5131   Unchanged:5131   awful    :  12  
                                                    Bad      :   7  
                                                    Unchanged:5112  
      oil_cat2         oil_cat3         oil_cat4         oil_cat5   
 awful    :  12   awful    :  12   awful    :  12   awful    :  12  
 Bad      :   7   Bad      :   7   Bad      :   7   Ba

As we can observe, most of the observations come within the -0.3 and 0.3 standard deviations (Unchanged). As such, for now, let us only consider oil prices to predict the price direction.

In [86]:
# Modelling using NaiveBayes
model.NB <- NaiveBayes(priceDir ~ oil_cat1 + oil_cat2 + oil_cat3 + oil_cat4 + oil_cat5 ,data=train)
test <- test[c('oil_cat1','oil_cat2','oil_cat3','oil_cat4','oil_cat5')]
#predictions <- predict(model.NB, test[])
#confusionMatrix(test$priceDir, predictions$class)
summary(test)

      oil_cat1         oil_cat2         oil_cat3         oil_cat4   
 awful    :   0   awful    :   0   awful    :   0   awful    :   0  
 Bad      :   6   Bad      :   6   Bad      :   9   Bad      :   7  
 Good     :   6   Good     :   8   Good     :  10   Good     :  13  
 Great    :   0   Great    :   0   Great    :   0   Great    :   0  
 Unchanged:1699   Unchanged:1697   Unchanged:1692   Unchanged:1691  
      oil_cat5   
 awful    :   0  
 Bad      :   9  
 Good     :  16  
 Great    :   0  
 Unchanged:1686  